# Validating Agent Output With FunctionTarget

## Setup and Imports

In [ ]:
# Copyright (c) 2023 - 2025, AG2ai, Inc., AG2ai open-source projects maintainers and core contributors
#
# SPDX-License-Identifier: Apache-2.0

import os
from typing import Any

from autogen import ConversableAgent, LLMConfig
from autogen.agentchat import initiate_group_chat
from autogen.agentchat.group import AgentNameTarget, ContextVariables, FunctionTarget, FunctionTargetResult, StayTarget
from autogen.agentchat.group.patterns import DefaultPattern

cfg = LLMConfig(api_type="openai", model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

ctx = ContextVariables(data={})

## Agent Definitions

In [ ]:
# Agents
job_application_agent = ConversableAgent(
    name="job_application_agent",
    llm_config=cfg,
    system_message="""
    You are a job application agent helping users draft compelling job application emails for tech roles.

    You will receive info about the specific job role that you are applying for.

    Start your email with the exact words 'Dear Stope Recruiting Team,' and end with 'Sincerely, [Your Name]'.
    Make the email highly compelling such that it would convince a senior recruiter to accept your application.
    """,
)

application_review_agent = ConversableAgent(
    name="application_review_agent",
    llm_config=cfg,
    system_message="""You are a senior recruiter reviewing job applications, working at the company Stope, a leader in GPU optimization for AI workloads.
    Review the application email provided to you and determine whether you would accept or reject the application based on its content.""",
)

## FunctionTarget definition

In [ ]:
# A FunctionTarget must have the signature (input: str, context_variables: Any) -> FunctionTargetResult
def validate_application(application: str, context_variables: Any) -> FunctionTargetResult:
    """
    Custom FunctionTarget to validate the application draft, ensuring it begins with exactly 'Dear Stope Recruiting Team', as instructed.
    If it does, proceed to the next agent for review; if not, explain the issue and ask for a revision.
    """
    # Save application to context
    context_variables["application"] = application

    # Simple validation example
    if "Dear Stope Recruiting Team" in application:
        return FunctionTargetResult(
            messages=f"Review the following job application: {application}",
            target=AgentNameTarget("application_review_agent"),
            context_variables=context_variables,
        )

    else:
        return FunctionTargetResult(
            messages="The application does not start with the required salutation 'Dear Stope Recruiting Team'. Please regenerate it, ensuring it starts with this exact phrase.",
            target=StayTarget(),
            context_variables=context_variables,
        )

## Pattern Configuration and Group Chat Initiation

In [ ]:
# Conversation pattern
pattern = DefaultPattern(
    initial_agent=job_application_agent,
    agents=[job_application_agent, application_review_agent],
    user_agent=None,
    context_variables=ctx,
    group_manager_args={"llm_config": cfg},
)

# Register after-work handoff
job_application_agent.handoffs.set_after_work(FunctionTarget(validate_application))

# Run
initiate_group_chat(
    pattern=pattern,
    messages="You are applying for the position of Software Engineer at Stope, a leader in GPU Optimization for AI workloads.",
    max_rounds=8,
)